In [54]:
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.similarities import MatrixSimilarity
from gensim import corpora, models
from scipy.spatial.distance import jaccard
from scipy.stats import entropy
import numpy as np
nlp = spacy.load("en_core_web_sm")

In [55]:
# Step 1: Create a small collection of documents
documents = [

  # Cosine similarity
  '''
  The cosine similarity metric measures the angle between two vectors, making it highly effective for comparing documents.
  By using TF-IDF representations, textual data can be transformed into feature vectors.
  Tools like Gensim make it easy to compute cosine similarity for ranking document relevance
  ''',
  # Text Rank
  ''' 
  The TextRank algorithm builds a graph of sentences to identify the most important ones based on their ranks. 
  It is often used for summarization and keyword extraction tasks in natural language processing. 
  By analyzing a graph, TextRank finds influential sentences or phrases for concise summaries.
  ''',
  # Cosine similarity
  '''
  When comparing documents, cosine similarity is one of the most commonly used metrics due to its efficiency. 
  Representing texts with TF-IDF vectors ensures that important terms are weighted properly in similarity calculations. 
  Libraries like Gensim streamline the use of cosine similarity for information retrieval tasks. 
  ''',
  # Text Rank
  ''' 
  In TextRank, sentences are treated as nodes connected by edges that represent their similarity. 
  This graph-based approach is widely used in keyword extraction and summarization. 
  By assigning importance to each node, TextRank ensures that the most significant content is retained.
  ''',
  # Cosine similarity
  ''' 
  The cosine similarity metric provides an effective way to evaluate how similar two documents are based on their content.
  Using TF-IDF vectors enhances this process by assigning higher weights to relevant terms.
  The ease of implementing cosine similarity with tools like Gensim makes it a practical choice for many applications.
  '''
]


#### Preprocessing

In [56]:
# Load SpaCy model
# Preprocessing function
def preprocess_text(text):
    doc = nlp(text)
    # Lemmatize, remove stop words, keep only alphabetic tokens
    texts = [token.lemma_ for token in nlp(doc) if token.is_alpha and not token.is_stop]
    return " ".join(texts)

# Apply preprocessing to documents
documents = [preprocess_text(doc) for doc in documents]


In [57]:
documents

['cosine similarity metric measure angle vector make highly effective compare document TF IDF representation textual datum transform feature vector tool like Gensim easy compute cosine similarity rank document relevance',
 'TextRank algorithm build graph sentence identify important one base rank summarization keyword extraction task natural language processing analyze graph TextRank find influential sentence phrase concise summary',
 'compare document cosine similarity commonly metric efficiency represent text TF IDF vector ensure important term weight properly similarity calculation library like Gensim streamline use cosine similarity information retrieval task',
 'TextRank sentence treat node connect edge represent similarity graph base approach widely keyword extraction summarization assign importance node TextRank ensure significant content retain',
 'cosine similarity metric provide effective way evaluate similar document base content TF IDF vector enhance process assign high weig

### Similarities calculation

* Step 4: Calculate Pairwise Similarities 
* Utiliser la cosine similarity pour mesurer la similitude entre les vecteurs. 
* Une valeur proche de 1 indique une forte similarité.

#### BoW

###### Gensims's Matrix Similarity

In [62]:
texts = [doc.lower().split() for doc in documents]
dictionary = corpora.Dictionary(texts)  # Create a dictionary
bow_corpus = [dictionary.doc2bow(text) for text in texts]  # Create BoW representation


bow_index = MatrixSimilarity(bow_corpus)  # Build the similarity index
bow_similarities = np.zeros((len(bow_corpus), len(bow_corpus)))

for i, vec in enumerate(bow_corpus):
    bow_similarities[i] = bow_index[vec]

np.fill_diagonal(bow_similarities, 1.0)
print("Pairwise Similarities - BoW (Gensim):")
print(bow_similarities)

Pairwise Similarities - BoW (Gensim):
[[1.         0.02906191 0.54054052 0.0632772  0.54799664]
 [0.02906191 1.         0.05812382 0.40824831 0.02946278]
 [0.54054052 0.05812382 1.         0.15819299 0.52059686]
 [0.0632772  0.40824831 0.15819299 1.         0.16037509]
 [0.54799664 0.02946278 0.5205968  0.16037509 1.        ]]


###### Cosine Similarity

In [63]:
# Step 3: Representations - Bag of Words (BoW), TF-IDF, and LDA
# BoW Representation : Transforme chaque document en un vecteur qui représente la fréquence brute des mots.
bow_vectorizer = CountVectorizer()
bow_matrix = bow_vectorizer.fit_transform(documents)

# BoW Similarities
bow_similarities = cosine_similarity(bow_matrix)

# Display Results
# Chaque cellule [i, j] représente la similarité cosine entre les documents i et j.
# Diagone : Les valeurs sont 1, car chaque document est parfaitement similaire à lui-même.
# Autres valeurs : Les similarités entre les documents dépendent du nombre de mots communs entre eux.
print("Pairwise Similarities - BoW:")
print(bow_similarities)

Pairwise Similarities - BoW:
[[1.         0.02906191 0.54054054 0.0632772  0.54799662]
 [0.02906191 1.         0.05812382 0.40824829 0.02946278]
 [0.54054054 0.05812382 1.         0.158193   0.52059679]
 [0.0632772  0.40824829 0.158193   1.         0.16037507]
 [0.54799662 0.02946278 0.52059679 0.16037507 1.        ]]


###### Jaccard Similarity

In [64]:
# Jaccard Similarity for BoW
def calculate_jaccard(matrix):
    binary_matrix = (matrix > 0).astype(int)
    n_docs = binary_matrix.shape[0]
    jaccard_sim = np.zeros((n_docs, n_docs))
    for i in range(n_docs):
        for j in range(n_docs):
            jaccard_sim[i, j] = 1 - jaccard(binary_matrix[i], binary_matrix[j])
    return jaccard_sim

np.fill_diagonal(bow_similarities, 1.0)
bow_jaccard_similarity = calculate_jaccard(bow_matrix.toarray())
print("\nBoW Jaccard Similarity:\n", bow_jaccard_similarity)


BoW Jaccard Similarity:
 [[1.         0.0212766  0.24390244 0.02222222 0.27906977]
 [0.0212766  1.         0.04255319 0.18918919 0.01923077]
 [0.24390244 0.04255319 1.         0.06818182 0.24444444]
 [0.02222222 0.18918919 0.06818182 1.         0.08510638]
 [0.27906977 0.01923077 0.24444444 0.08510638 1.        ]]


#### TF-IDF

###### Gensim's Matrix Similarity

In [65]:
# TF-IDF using Gensim
tfidf_model = models.TfidfModel(bow_corpus)  # Train TF-IDF model
tfidf_corpus = [tfidf_model[doc] for doc in bow_corpus]  # Transform BoW corpus to TF-IDF

# TF-IDF Similarities with Gensim
tfidf_index = MatrixSimilarity(tfidf_corpus)  # Build the similarity index
tfidf_similarities = np.zeros((len(tfidf_corpus), len(tfidf_corpus)))

for i, vec in enumerate(tfidf_corpus):
    tfidf_similarities[i] = tfidf_index[vec]

print("\nPairwise Similarities - TF-IDF (Gensim):")
print(tfidf_similarities)


Pairwise Similarities - TF-IDF (Gensim):
[[1.         0.01962236 0.12517105 0.00250597 0.15272599]
 [0.01962236 1.         0.04047105 0.2084666  0.00568988]
 [0.12517105 0.04047105 1.         0.04745131 0.12497157]
 [0.00250597 0.2084666  0.04745131 1.         0.04788698]
 [0.15272601 0.00568988 0.12497157 0.04788698 1.        ]]


###### Cosine Similarity

In [24]:
# TF-IDF Representation : Ajoute une pondération basée sur l'importance des mots dans le corpus.
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# TF-IDF Similarities
tfidf_similarities = cosine_similarity(tfidf_matrix)

# Avec TF-IDF : Les similarités sont plus faibles car la pondération TF-IDF réduit l'importance des mots fréquents.
# TF-IDF capture mieux les termes significatifs dans le contexte.

print("\nPairwise Similarities - TF-IDF:")
print(tfidf_similarities)


Pairwise Similarities - TF-IDF:
[[1.         0.02696877 0.36870828 0.02863642 0.38242601]
 [0.02696877 1.         0.0548972  0.32919482 0.01817173]
 [0.36870828 0.0548972  1.         0.10349062 0.34996995]
 [0.02863642 0.32919482 0.10349062 1.         0.10521584]
 [0.38242601 0.01817173 0.34996995 0.10521584 1.        ]]


###### Jaccard Similarity

In [49]:
def calculate_jaccard(matrix):
    binary_matrix = (matrix > 0).astype(int)
    n_docs = binary_matrix.shape[0]
    jaccard_sim = np.zeros((n_docs, n_docs))
    for i in range(n_docs):
        for j in range(n_docs):
            jaccard_sim[i, j] = 1 - jaccard(binary_matrix[i], binary_matrix[j])
    return jaccard_sim

tfidf_jaccard_similarity = calculate_jaccard(tfidf_matrix.toarray())
print("Tf-idf Jaccard Similarity:\n", tfidf_jaccard_similarity)

Tf-idf Jaccard Similarity:
 [[1.         0.0212766  0.24390244 0.02222222 0.27906977]
 [0.0212766  1.         0.04255319 0.18918919 0.01923077]
 [0.24390244 0.04255319 1.         0.06818182 0.24444444]
 [0.02222222 0.18918919 0.06818182 1.         0.08510638]
 [0.27906977 0.01923077 0.24444444 0.08510638 1.        ]]


#### LDA

###### Gensim's Matrix Similarity

In [72]:
# LDA Representation : Modélise chaque document comme une distribution sur des "thèmes" (topics).
#                      Chaque thème est lui-même une distribution de mots.
texts = [doc.lower().split() for doc in documents]  # Tokenize
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda_model = models.LdaModel(corpus, num_topics=2, id2word=dictionary, passes=50)
lda_topics = [lda_model[doc] for doc in corpus]

for idx, topic in lda_model.print_topics(num_topics=2, num_words=7):
    print(f"Topic {idx}: {topic}")

Topic 0: 0.057*"similarity" + 0.049*"cosine" + 0.034*"vector" + 0.034*"document" + 0.026*"metric" + 0.026*"idf" + 0.026*"like"
Topic 1: 0.049*"textrank" + 0.038*"sentence" + 0.038*"graph" + 0.027*"base" + 0.027*"summarization" + 0.027*"extraction" + 0.027*"keyword"


In [73]:
# LDA Similarities (based on topic distributions) : Mesure la similarité entre les distributions des thèmes des documents.
index = MatrixSimilarity(lda_model[corpus])
lda_similarities = np.zeros((len(corpus), len(corpus)))

for i, topic1 in enumerate(lda_model[corpus]):
    lda_similarities[i] = index[topic1]

np.fill_diagonal(bow_similarities, 1.0)

In [74]:
print("\nPairwise Similarities - LDA:")
print(lda_similarities)


Pairwise Similarities - LDA:
[[1.         0.03768451 0.9999994  0.04244778 0.99999964]
 [0.03768569 1.         0.03872294 0.99998868 0.03686775]
 [0.99999946 0.03872273 1.         0.0434858  0.99999827]
 [0.042446   0.99998862 0.04348305 1.         0.04162822]
 [0.99999964 0.03687339 0.99999821 0.04163681 0.99999994]]


###### KL Divergence

In [71]:
# KL Divergence for LDA
lda_distributions = np.array([
    [dict(topic).get(i, 0) for i in range(lda_model.num_topics)] 
    for topic in lda_topics
])

def calculate_kl_divergence(distributions):
    n_docs = distributions.shape[0]
    kl_sim = np.zeros((n_docs, n_docs))
    for i in range(n_docs):
        for j in range(n_docs):
            kl_sim[i, j] = entropy(distributions[i] + 1e-12, distributions[j] + 1e-12)
    return kl_sim

lda_kl_similarity = calculate_kl_divergence(lda_distributions)

print("\nLDA KL Divergence Similarity:")
for row in lda_kl_similarity:
    formatted_row = ["{:.2f}".format(value) for value in row]  # Format to 2 decimal places
    print("[{}]".format(" ".join(formatted_row)))


LDA KL Divergence Similarity:
[0.00 3.78 0.00 3.58 0.00]
[3.87 0.00 3.81 0.00 3.91]
[0.00 3.78 0.00 3.57 0.00]
[3.83 0.00 3.78 0.00 3.88]
[0.00 3.79 0.00 3.58 0.00]


In [ ]:
'''
BoW (Bag of Words) :
Les similarités ici sont relativement faibles, sauf entre certains documents (par exemple, [0,2] et [0,4]).
 Cela est attendu car BoW se concentre uniquement sur la fréquence brute des mots sans considérer leur importance ou 
 le contexte. Les similarités élevées (ex. [0,2]) sont dues à des termes partagés fréquents. 
 C'est simple mais souvent imprécis pour capturer des nuances sémantiques.

TF-IDF (Term Frequency-Inverse Document Frequency) :
TF-IDF produit des similarités plus faibles que BoW, indiquant une pondération plus fine basée sur l'importance 
relative des mots (fréquence locale et rareté globale). Cela réduit l'effet des mots communs dans plusieurs documents
(ex. "the", "is"). Cela améliore la précision pour des documents ayant des mots-clés significatifs, mais reste limité
pour des concepts complexes.

LDA (Latent Dirichlet Allocation) :
Les similarités avec LDA sont très élevées (proches de 1) entre presque tous les documents. 
Cela s'explique par la capacité de LDA à capturer les thématiques sous-jacentes plutôt que de se limiter aux mots exacts.
Cependant, ces similarités excessivement élevées peuvent indiquer un sur-apprentissage ou un modèle mal ajusté.

TF-IDF est généralement plus précis pour des données générales ou peu structurées car il gère les mots-clés 
avec plus de nuance.

LDA est meilleur pour les relations sémantiques profondes (par exemple, "chien" et "animal"), 
mais il dépend fortement du réglage des paramètres (nombre de topics, etc.).

BoW est simple et utile pour des tâches rudimentaires, mais souvent trop limité.
'''

In [ ]:
''' 

Bag of Words (BoW) :
Avantages :
    Simple à implémenter.
    Peu coûteux en calculs.
    Efficace pour des tâches où le vocabulaire est limité.
Limites :
    Ignore l'ordre des mots et le contexte.
    Insensible aux synonymes ou relations sémantiques.
    Sensible aux mots fréquents sans importance.
TF-IDF :
Avantages :
    Pondère l'importance des mots en fonction de leur rareté dans le corpus.
    Réduit l'impact des mots "stopwords".
    Efficace pour capturer des mots significatifs.
Limites :
    Toujours basé sur des mots exacts (ne capture pas de sémantique profonde).
    Moins performant pour de grands corpus complexes.
LDA :
Avantages :
    Modélise les thèmes latents pour capturer des relations sémantiques entre mots.
    Insensible aux mots exacts : regroupe des documents partageant des thématiques similaires.
    Convient pour des corpus larges et variés.
Limites :
    Complexité computationnelle plus élevée.
    Sensible au choix des hyperparamètres (ex. nombre de topics).
    Peut produire des similarités biaisées si mal entraîné.

'''

In [ ]:
'''  
LDA est un modèle probabiliste qui représente les documents comme une combinaison de thèmes (topics),
chaque thème étant une distribution probabiliste sur les mots. Voici comment il capture des relations sémantiques :

Thématiques communes :
LDA regroupe des mots souvent utilisés ensemble dans différents documents (ex. "chien", "chat", "animal") 
sous un même thème, même s'ils n'apparaissent pas dans un même document.

Probabilités partagées :
Si deux documents sont composés majoritairement des mêmes thèmes, ils sont considérés comme similaires, 
même si leurs mots exacts diffèrent.

Réduction de bruit :
En modélisant des distributions latentes, LDA ignore les mots fréquents mais peu significatifs (stopwords).

Contextualisation des mots :
Les thèmes définis par LDA peuvent capturer des concepts abstraits (ex. "finance" pour "banque", "argent"), 
reliant ainsi des documents en fonction de leur sémantique plutôt que leur lexique exact.

'''